In [1]:
from data_processing import process_mat_file
import numpy as np
import pandas as np 
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
input_data_path = "/Users/cooper/Desktop/climate-impacts/data/Input_data.mat"
path_exp_first = '/Users/cooper/Desktop/climate-impacts/data/Exponential/Seperated_FirstOrder_Exp_1500(30).mat'

In [3]:
present_data = process_mat_file(input_data_path)
exp_first_data = process_mat_file(path_exp_first)

In [4]:
exp_first_data.head(61)

,Precipitation,T_max,T_min,T_avg,Year,Month,Day,Simulation
0,1.052159,0.575836,-3.966348,-1.695256,1980,Jan,1,1
1,2.840043,2.131566,-2.196253,-0.032343,1980,Jan,2,1
2,0.000000,0.337740,-2.734396,-1.198328,1980,Jan,3,1
3,0.000000,4.614041,0.185355,2.399698,1980,Jan,4,1
4,2.044771,2.277004,-3.072299,-0.397647,1980,Jan,5,1
...,...,...,...,...,...,...,...,...
56,0.000000,6.088176,-4.696267,0.695955,1980,Feb,26,1
57,3.640961,0.331758,-6.676655,-3.172448,1980,Feb,27,1
58,0.000000,5.267578,-3.819374,0.724102,1980,Feb,28,1
59,0.000000,3.421929,-4.461357,-0.519714,1980,Mar,1,1
